In [6]:
%pip install -q cloud-tpu-client https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.13-cp38-cp38m-linux_x86_64.whl
%pip install -q hydrology polars neuralforecast wandb

Note: you may need to restart the kernel to use updated packages.


ERROR: torch_xla-1.13-cp38-cp38m-linux_x86_64.whl is not a supported wheel on this platform.


Note: you may need to restart the kernel to use updated packages.


In [7]:
from hydrology import HydrologyApi, Measure
from datetime import datetime
import polars as pl

## Load Dataset

In [80]:
api = HydrologyApi()

level_stations = api.get_stations(Measure.MeasureType.LEVEL, river="River Wear")
rainfall_stations = api.get_stations(
    Measure.MeasureType.RAINFALL, 
    position=(54.774, -1.558), radius=15
).filter(
    ~pl.col("station_name").is_in(
        # Stations with lots of missing data
        [
            "ESH Winning",
            "Stanley Hustledown",
            "Washington",
        ]
    )
)

measures = [
    Measure(station_id, Measure.MeasureType.LEVEL)
    for station_id in level_stations["station_id"]
] + [
    Measure(station_id, Measure.MeasureType.RAINFALL)
    for station_id in rainfall_stations["station_id"]
]

stations = pl.concat(
    [
        level_stations,
        rainfall_stations,
    ],
).unique()

df = api.get_measures(measures, stations, start_date=datetime(2007, 1, 1))
train_split = 0.8
train_records = int(len(df) * train_split)

train_df = df.slice(0, train_records)
test_df = df.slice(train_records)

train_df.tail()

params = ({'observedProperty': ['waterLevel'], 'riverName': 'River Wear', 'status.label': 'Active'},)
params = ({'observedProperty': ['rainfall'], 'lat': 54.774, 'long': -1.558, 'dist': 15, 'status.label': 'Active'},)


timestamp,Durham New Elvet Bridge level-i-900-m,North Dalton rainfall-t-900-mm,Sunderland Bridge level-i-900-m,Chester Le Street level-i-900-m,Knitlsey Mill rainfall-t-900-mm,Witton Park level-i-900-m,Peterlee rainfall-t-900-mm,Evenwood Gate rainfall-t-900-mm,Fulwell rainfall-t-900-mm,Stanhope level-i-900-m,Tunstall rainfall-t-900-mm,Harpington Hill Farm rainfall-t-900-mm
datetime[μs],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2021-01-06 02:45:00,0.752,0.0,0.703,0.857,0.0,0.635,0.0,0.0,0.0,0.391,0.0,0.0
2021-01-06 03:00:00,0.75,0.0,0.701,0.855,0.0,0.635,0.0,0.0,0.0,0.392,0.0,0.2
2021-01-06 03:15:00,0.745,0.0,0.699,0.854,0.0,0.634,0.0,0.0,0.0,0.391,0.0,0.2
2021-01-06 03:30:00,0.745,0.0,0.696,0.855,0.0,0.635,0.0,0.0,0.0,0.391,0.0,0.0
2021-01-06 03:45:00,0.743,0.0,0.695,0.855,0.0,0.634,0.0,0.0,0.0,0.391,0.0,0.0


## Feature Engineering

In [82]:
from math import pi

start_year = 2007

time_features = (
    train_df
    .select(pl.col("timestamp"))
    .with_columns(
        (pl.col("timestamp").dt.ordinal_day() / 365).alias("day_of_year"),
        ((pl.col("timestamp").dt.epoch(time_unit='s') - datetime(start_year, 1, 1).timestamp()) / (60 * 60 * 24 * 365.2524)).alias(f"years_since_{start_year}"),
    )
    .select(
      (pl.col("day_of_year") * 2 * pi).cos().alias("cos_day_of_year"),
      (pl.col("day_of_year") * 2 * pi).sin().alias("sin_day_of_year"),
       pl.col(f"years_since_{start_year}")
    )
)

train_df = pl.concat([train_df, time_features], how='horizontal')
train_df.head()

timestamp,Durham New Elvet Bridge level-i-900-m,North Dalton rainfall-t-900-mm,Sunderland Bridge level-i-900-m,Chester Le Street level-i-900-m,Knitlsey Mill rainfall-t-900-mm,Witton Park level-i-900-m,Peterlee rainfall-t-900-mm,Evenwood Gate rainfall-t-900-mm,Fulwell rainfall-t-900-mm,Stanhope level-i-900-m,Tunstall rainfall-t-900-mm,Harpington Hill Farm rainfall-t-900-mm,cos_day_of_year,sin_day_of_year,years_since_2007
datetime[μs],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64
2007-01-01 00:00:00,0.726,0.0,0.851,0.824,0.0,1.009,0.0,0.0,0.0,0.859,0.0,0.0,0.999852,0.017213,0.0
2007-01-01 00:15:00,0.73,0.0,0.863,0.821,0.0,0.997,0.0,0.0,0.0,0.855,0.0,0.0,0.999852,0.017213,0.000029
2007-01-01 00:30:00,0.74,0.0,0.876,0.823,0.0,1.0,0.0,0.0,0.0,0.845,0.0,0.0,0.999852,0.017213,0.000057
2007-01-01 00:45:00,0.744,0.0,0.886,0.819,0.0,1.001,0.0,0.0,0.0,0.826,0.0,0.0,0.999852,0.017213,0.000086
2007-01-01 01:00:00,0.763,0.0,0.894,0.823,0.0,0.993,0.0,0.0,0.0,0.825,0.0,0.0,0.999852,0.017213,0.000114


In [117]:
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, MinMaxScaler

X_preprocessing = make_column_transformer(
      (
          StandardScaler(),
          make_column_selector(pattern='level-i-900-m'),
      ),
      (
          MinMaxScaler(),
          make_column_selector(pattern='rainfall-t-900-mm'),
      ),
      remainder='passthrough'
)

y_preprocessing = StandardScaler()

X_train = train_df.select(pl.col("*").exclude("timestamp", "Durham New Elvet Bridge level-i-900-m")).to_pandas()
y_train = train_df.select("Durham New Elvet Bridge level-i-900-m").to_numpy()

X_train = X_preprocessing.fit_transform(X_train)
y_train = y_preprocessing.fit_transform(y_train)

X_train.shape, y_train.shape

((491440, 14), (491440, 1))

In [118]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

BATCH_SIZE = 64

class TimeSeriesDataset(Dataset):
    def __init__(
        self, 
        X_past: np.array,
        y: np.array,
        seq_length: int = 4 * 24,
        pred_length: int = 4 * 6,
    ):
        self.X_past = X_past
        self.y = y
        self.seq_length = seq_length
        self.pred_length = pred_length

    def __len__(self):
        return len(self.X_past) - self.seq_length - self.pred_length + 1

    def __getitem__(self, idx):
        return (
            self.X_past[idx:idx+self.seq_length],
            self.y[idx+self.seq_length:idx+self.seq_length+self.pred_length]
        )
        
dataset = TimeSeriesDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
import torch
from torch import nn
import math


class Transpose(nn.Module):
    def __init__(self, *dims, contiguous=False):
        super().__init__()
        self.dims, self.contiguous = dims, contiguous

    def forward(self, x):
        if self.contiguous:
            return x.transpose(*self.dims).contiguous()
        else:
            return x.transpose(*self.dims)


def get_activation_fn(activation):
    if callable(activation):
        return activation()
    elif activation.lower() == 'relu':
        return nn.ReLU()
    elif activation.lower() == 'gelu':
        return nn.GELU()
    raise ValueError(
        f'{activation} is not available. You can use "relu", "gelu", or a callable'
    )


# decomposition


class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """

    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """

    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean


# pos_encoding


def PositionalEncoding(q_len, d_model, normalize=True):
    pe = torch.zeros(q_len, d_model)
    position = torch.arange(0, q_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    if normalize:
        pe = pe - pe.mean()
        pe = pe / (pe.std() * 10)
    return pe


SinCosPosEncoding = PositionalEncoding


def Coord2dPosEncoding(
    q_len, d_model, exponential=False, normalize=True, eps=1e-3, verbose=False
):
    x = 0.5 if exponential else 1
    i = 0
    for i in range(100):
        cpe = (
            2
            * (torch.linspace(0, 1, q_len).reshape(-1, 1) ** x)
            * (torch.linspace(0, 1, d_model).reshape(1, -1) ** x)
            - 1
        )
        if abs(cpe.mean()) <= eps:
            break
        elif cpe.mean() > eps:
            x += 0.001
        else:
            x -= 0.001
        i += 1
    if normalize:
        cpe = cpe - cpe.mean()
        cpe = cpe / (cpe.std() * 10)
    return cpe


def Coord1dPosEncoding(q_len, exponential=False, normalize=True):
    cpe = (
        2 * (torch.linspace(0, 1, q_len).reshape(-1, 1) ** (0.5 if exponential else 1))
        - 1
    )
    if normalize:
        cpe = cpe - cpe.mean()
        cpe = cpe / (cpe.std() * 10)
    return cpe


def positional_encoding(pe, learn_pe, q_len, d_model):
    # Positional encoding
    if pe == None:
        W_pos = torch.empty(
            (q_len, d_model)
        )  # pe = None and learn_pe = False can be used to measure impact of pe
        nn.init.uniform_(W_pos, -0.02, 0.02)
        learn_pe = False
    elif pe == 'zero':
        W_pos = torch.empty((q_len, 1))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'zeros':
        W_pos = torch.empty((q_len, d_model))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'normal' or pe == 'gauss':
        W_pos = torch.zeros((q_len, 1))
        torch.nn.init.normal_(W_pos, mean=0.0, std=0.1)
    elif pe == 'uniform':
        W_pos = torch.zeros((q_len, 1))
        nn.init.uniform_(W_pos, a=0.0, b=0.1)
    elif pe == 'lin1d':
        W_pos = Coord1dPosEncoding(q_len, exponential=False, normalize=True)
    elif pe == 'exp1d':
        W_pos = Coord1dPosEncoding(q_len, exponential=True, normalize=True)
    elif pe == 'lin2d':
        W_pos = Coord2dPosEncoding(q_len, d_model, exponential=False, normalize=True)
    elif pe == 'exp2d':
        W_pos = Coord2dPosEncoding(q_len, d_model, exponential=True, normalize=True)
    elif pe == 'sincos':
        W_pos = PositionalEncoding(q_len, d_model, normalize=True)
    else:
        raise ValueError(f"{pe} is not a valid pe (positional encoder. Available types: 'gauss'=='normal', \
        'zeros', 'zero', uniform', 'lin1d', 'exp1d', 'lin2d', 'exp2d', 'sincos', None.)")
    return nn.Parameter(W_pos, requires_grad=learn_pe)
  
class RevIN(nn.Module):
  def __init__(self, num_features: int, eps=1e-5, affine=True, subtract_last=False):
      """
      :param num_features: the number of features or channels
      :param eps: a value added for numerical stability
      :param affine: if True, RevIN has learnable affine parameters
      """
      super(RevIN, self).__init__()
      self.num_features = num_features
      self.eps = eps
      self.affine = affine
      self.subtract_last = subtract_last
      if self.affine:
          self._init_params()

  def forward(self, x, mode: str):
      if mode == 'norm':
          self._get_statistics(x)
          x = self._normalize(x)
      elif mode == 'denorm':
          x = self._denormalize(x)
      else:
          raise NotImplementedError
      return x

  def _init_params(self):
      # initialize RevIN params: (C,)
      self.affine_weight = nn.Parameter(torch.ones(self.num_features))
      self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

  def _get_statistics(self, x):
      dim2reduce = tuple(range(1, x.ndim - 1))
      if self.subtract_last:
          self.last = x[:, -1, :].unsqueeze(1)
      else:
          self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
      self.stdev = torch.sqrt(
          torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps
      ).detach()

  def _normalize(self, x):
      if self.subtract_last:
          x = x - self.last
      else:
          x = x - self.mean
      x = x / self.stdev
      if self.affine:
          x = x * self.affine_weight
          x = x + self.affine_bias
      return x

  def _denormalize(self, x):
      if self.affine:
          x = x - self.affine_bias
          x = x / (self.affine_weight + self.eps * self.eps)
      x = x * self.stdev
      if self.subtract_last:
          x = x + self.last
      else:
          x = x + self.mean
      return x


In [ ]:

class PatchMixerLayer(nn.Module):
    def __init__(self, dim, a, kernel_size=8):
        super().__init__()
        self.Resnet = nn.Sequential(
            nn.Conv1d(dim, dim, kernel_size=kernel_size, groups=dim, padding='same'),
            nn.GELU(),
            nn.BatchNorm1d(dim),
        )
        self.Conv_1x1 = nn.Sequential(
            nn.Conv1d(dim, a, kernel_size=1), nn.GELU(), nn.BatchNorm1d(a)
        )

    def forward(self, x):
        x = x + self.Resnet(x)  # x: [batch * n_val, patch_num, d_model]
        x = self.Conv_1x1(x)  # x: [batch * n_val, a, d_model]
        return x


class Model(nn.Module):
    def __init__(self, configs):
        super().__init__()
        self.model = Backbone(configs)

    def forward(self, x):
        x = self.model(x)
        return x


class Backbone(nn.Module):
    def __init__(
        self, 
        configs, 
        revin=True, 
        affine=True, 
        subtract_last=False
    ):
        super().__init__()

        self.nvals = configs.enc_in
        self.lookback = configs.seq_len
        self.forecasting = configs.pred_len
        self.patch_size = configs.patch_len
        self.stride = configs.stride
        self.kernel_size = configs.mixer_kernel_size

        self.PatchMixer_blocks = nn.ModuleList([])
        self.padding_patch_layer = nn.ReplicationPad1d((0, self.stride))
        self.patch_num = int((self.lookback - self.patch_size) / self.stride + 1) + 1
        # if configs.a < 1 or configs.a > self.patch_num:
        #     configs.a = self.patch_num
        self.a = self.patch_num
        self.d_model = configs.d_model
        self.dropout = configs.dropout
        self.head_dropout = configs.head_dropout
        self.depth = configs.e_layers
        for _ in range(self.depth):
            self.PatchMixer_blocks.append(
                PatchMixerLayer(
                    dim=self.patch_num, a=self.a, kernel_size=self.kernel_size
                )
            )
        self.W_P = nn.Linear(self.patch_size, self.d_model)
        self.head0 = nn.Sequential(
            nn.Flatten(start_dim=-2),
            nn.Linear(self.patch_num * self.d_model, self.forecasting),
            nn.Dropout(self.head_dropout),
        )
        self.head1 = nn.Sequential(
            nn.Flatten(start_dim=-2),
            nn.Linear(self.a * self.d_model, int(self.forecasting * 2)),
            nn.GELU(),
            nn.Dropout(self.head_dropout),
            nn.Linear(int(self.forecasting * 2), self.forecasting),
            nn.Dropout(self.head_dropout),
        )
        self.dropout = nn.Dropout(self.dropout)
        # RevIn
        self.revin = revin
        if self.revin:
            self.revin_layer = RevIN(
                self.nvals, affine=affine, subtract_last=subtract_last
            )

    def forward(self, x):
        bs = x.shape[0]
        nvars = x.shape[-1]
        if self.revin:
            x = self.revin_layer(x, 'norm')
        x = x.permute(0, 2, 1)  # x: [batch, n_val, seq_len]

        x_lookback = self.padding_patch_layer(x)
        x = x_lookback.unfold(
            dimension=-1, size=self.patch_size, step=self.stride
        )  # x: [batch, n_val, patch_num, patch_size]

        x = self.W_P(x)  # x: [batch, n_val, patch_num, d_model]
        x = torch.reshape(
            x, (x.shape[0] * x.shape[1], x.shape[2], x.shape[3])
        )  # x: [batch * n_val, patch_num, d_model]
        x = self.dropout(x)
        u = self.head0(x)

        for PatchMixer_block in self.PatchMixer_blocks:
            x = PatchMixer_block(x)
        x = self.head1(x)
        x = u + x
        x = torch.reshape(x, (bs, nvars, -1))  # x: [batch, n_val, pred_len]
        x = x.permute(0, 2, 1)
        if self.revin:
            x = self.revin_layer(x, 'denorm')
        return x
